# FormDiagram

In this first part of the tutorial, we will set up the FormDiagram.
We will create the geometry from a `meshgrid`, identify the supports, relax the unsupported boundaries, and, finally, update the boundary conditions needed for the TNA solver.

First, the relevant imports.

In [1]:
import pathlib

from compas_notebook.viewer import Viewer

import compas
from compas_tna.diagrams import FormDiagram
from compas_tna.equilibrium import relax_boundary_openings


## Create a FormDiagram from a meshgrid

Creating a diagram from a grid is super easy.
Just set the dimensions in x and y, and specify the number of grid cells in each directions.
If the grid is symmetrical, you only have to specify the properties in the x direction...

In [2]:
form: FormDiagram = FormDiagram.from_meshgrid(dx=10, nx=10)  # type: ignore

## Identify the supports

The corner vertices of the meshgrid are going to be the supports of the vault.
In a quadrilateral grid, like the meshgrid, the vertices in the corners have `degree == 2`.

In [3]:
corners: list[int] = list(form.vertices_where(vertex_degree=2))  # type: ignore
form.vertices_attribute("is_support", True, keys=corners)

## Unsupported boundaries

Next, we relax the unsupported boundaries in between the supports to give them an initial curvature that will allow them to resist the outward thrusting forces coming from the inner part of the vault, and guide them to the supports.

In [4]:
form.edges_attribute("q", 10.0, keys=form.edges_on_boundary())
relax_boundary_openings(form, corners)

## Boundary conditions

Finally, we update the boundary conditions of the diagram to make it ready for TNA-based form finding.

In [5]:
form.update_boundaries()

## Export

Before visualising the result, we will export the FormDiagram in a form finding session, so it can be used as a starting point for the following steps in the process.

Note that there is no reliable way to identify the parent folder of the current notebook programmatically.
Therefore, the filepath below is hard-coded.
You should update it to whatever makes sense on your system.

In [ ]:
session = {"formdiagram": form, "forcediagram": None}
filepath = pathlib.Path().cwd() / "session.json"

compas.json_dump(session, filepath)

## Visualisation

In [7]:
viewer = Viewer()
viewer.scene.add(form)
viewer.show()